In [13]:
import os
import cv2
import numpy as np
from tqdm import tqdm # progress bar

REBUILD_DATA = False

In [11]:
class PetClassification:
    IMG_SIZE = 50
    CATS = "../data/PetImages/Cat"
    DOGS = "../data/PetImages/Dog"
    LABELS = {CATS:0, DOGS:1}
    training_data = []
    catcount = 0
    dogcount = 0
    
    # import training data into NP array, 
    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label, f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])

                    if (label == self.CATS):
                        self.catcount += 1
                    elif (label == self.DOGS):
                        self.dogcount += 1
                except Exception as e:
                    # in case corrupted data
                    pass
        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        print("Cats: ", self.catcount, " Dogs: ", self.dogcount)        

In [12]:
if REBUILD_DATA:
    pc = PetClassification()
    pc.make_training_data()

  0%|          | 56/12501 [00:00<00:22, 550.62it/s]

../data/PetImages/Cat


  0%|          | 49/12501 [00:00<00:25, 479.97it/s]

../data/PetImages/Dog


100%|██████████| 12501/12501 [00:21<00:00, 587.08it/s]


Cats:  12476  Dogs:  12470


In [15]:
training_data = np.load("training_data.npy", allow_pickle=True)